# 라이브러리

In [1]:
import re,pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm
from datetime import timedelta
from konlpy.tag import *

from sklearn.metrics import accuracy_score, f1_score, mean_squared_error, r2_score, confusion_matrix, classification_report, log_loss
from sklearn.pipeline import Pipeline 
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
# 경고 무시
import warnings
warnings.filterwarnings('ignore')

# 종목 선택, 뉴스 & 토론방 & 유튜브 데이터 통합

In [3]:
# LG화학, 삼성SDI, SK이노베이션, 고려아연, 포스코케미칼
stock_name = '삼성SDI'

In [4]:
naver_news = pd.read_csv('./data/refined_naver_news.csv', index_col=0)
daum_news = pd.read_csv('./data/refined_daum_news.csv', index_col=0)
naver_talks = pd.read_csv(f'./data/refined_naver_talks_{stock_name}.csv', index_col=0)
daum_talks = pd.read_csv(f'./data/refined_daum_talks_{stock_name}.csv', index_col=0)
youtube = pd.read_csv(f'./data/refined_youtube_{stock_name}.csv', index_col=0)

In [5]:
# 데이터 통합
news_df = pd.concat([naver_news, daum_news, naver_talks, daum_talks ,youtube])

# 'Date' 타입이 int 이므로 datetime으로 변환
news_df['Date'] = pd.to_datetime(news_df['Date'].astype(str))

# 합쳐진 데이터들의 인덱스 재설정
news_df.sort_values('Date', ignore_index=True, inplace=True)

In [6]:
# 2021년 주식 데이터가 1월 4일부터 있어서 슬라이싱
news_df = news_df[news_df[news_df['Date']== '2021-01-04'].index[0] : ]
news_df.head(2)

,Date,Title
639,2021-01-04,비올 일본 최대 병원체인과 실펌엑스 총판계약 체결
640,2021-01-04,환율 하락 전환 1086 2 감소0 1원


# 주가 데이터

In [7]:
stock_df = pd.read_csv(f'./data/{stock_name}_주가_데이터.csv', usecols = ['일자','등락률'])
stock_df['일자'] = pd.to_datetime(stock_df['일자'])
stock_df.head(2)

,일자,등락률
0,2021-01-04,6.85
1,2021-01-05,2.24


In [8]:
start = str(stock_df.iloc[0, 0])
end = str(stock_df.iloc[-1, 0])
print(start)
print(end)

2021-01-04 00:00:00
2022-06-30 00:00:00


# 데이터 프레임 합치기

In [9]:
## 뉴스일자 조정(예측대상(주가)의 일자와 맞추기 위해)
news_df['일자'] = news_df['Date'] + timedelta(days=1)

In [10]:
df = news_df.merge(stock_df)
df.columns = [df.columns[0], df.columns[1], '주가의 날짜', '등락률' ]
df.drop_duplicates('Title', inplace = True, ignore_index = True)  # 기사제목 중복 제거
df['Title'] = df['Title'].astype(str)
print(len(df))
df.head()

429241


,Date,Title,주가의 날짜,등락률
0,2021-01-04,비올 일본 최대 병원체인과 실펌엑스 총판계약 체결,2021-01-05,2.24
1,2021-01-04,환율 하락 전환 1086 2 감소0 1원,2021-01-05,2.24
2,2021-01-04,코스피 1 03p 0 04 오른 2874 50 출발 원 달러 환율 1 2원 ...,2021-01-05,2.24
3,2021-01-04,韓증시 사상최고치 시작 개장식선 안정적 시장운영에 방점,2021-01-05,2.24
4,2021-01-04,SK바이오팜 아벨 지분 매각으로 5천500만 달러 자본이득,2021-01-05,2.24


In [11]:
# multi 분류
stock_df['updown'] = 0

stock_df.loc[stock_df.query('등락률 > 1').index, 'updown'] = 1
stock_df.loc[stock_df.query('등락률 < -1').index, 'updown'] = -1

stock_df['updown'].value_counts()

 0    149
 1    111
-1    109
Name: updown, dtype: int64

In [32]:
# 뉴스 일자 열 추가 : 예측대상(= 익일의 주가)의 일자와 맞추기 위함

news_df['일자'] = news_df['Date'] + timedelta(days = 1)
news_df.head(3)

,Date,Title,일자
639,2021-01-04,비올 일본 최대 병원체인과 실펌엑스 총판계약 체결,2021-01-05
640,2021-01-04,환율 하락 전환 1086 2 감소0 1원,2021-01-05
641,2021-01-04,코스피 1 03p 0 04 오른 2874 50 출발 원 달러 환율 1 2원 ...,2021-01-05


# 뉴스 데이터 & 주가 데이터 합치기

In [12]:
df = news_df.merge(stock_df)

# df 칼럼명 : Date, Title, 주가의 날짜(구 : 일자), 등락률(y), updown
df.columns = [df.columns[0], df.columns[1], '주가의 날짜', '등락률(y)', 'updown']

# 기사 제목 중복 제거
df.drop_duplicates('Title', inplace = True, ignore_index = True)
df['Title'] = df['Title'].astype(str)
print(len(df))
df.head(3)

429241


,Date,Title,주가의 날짜,등락률(y),updown
0,2021-01-04,비올 일본 최대 병원체인과 실펌엑스 총판계약 체결,2021-01-05,2.24,1
1,2021-01-04,환율 하락 전환 1086 2 감소0 1원,2021-01-05,2.24,1
2,2021-01-04,코스피 1 03p 0 04 오른 2874 50 출발 원 달러 환율 1 2원 ...,2021-01-05,2.24,1


# 감성분석 모델 적용

## 감성사전 만들기

In [13]:
df.isnull().sum()

Date      0
Title     0
주가의 날짜    0
등락률(y)    0
updown    0
dtype: int64

In [14]:
df = df.dropna()
df.head(3)

,Date,Title,주가의 날짜,등락률(y),updown
0,2021-01-04,비올 일본 최대 병원체인과 실펌엑스 총판계약 체결,2021-01-05,2.24,1
1,2021-01-04,환율 하락 전환 1086 2 감소0 1원,2021-01-05,2.24,1
2,2021-01-04,코스피 1 03p 0 04 오른 2874 50 출발 원 달러 환율 1 2원 ...,2021-01-05,2.24,1


###  (필요시) Title 에서 종목명 제거하기

In [36]:
# stopstocks = pd.read_csv("./data/종목명.csv", encoding = 'cp949')
# stopstocks = stopstocks['한글 종목약명'].tolist()

# for i in tqdm(range(len(df['Title']))) :
#     for stopstock in stopstocks :
#         if stopstock in str(df['Title'][i]) :
#             df['Title'][i] = str(df['Title'][i]).replace(stopstock,'')



# 감성 사전 구축

In [15]:
okt = Okt()

In [16]:
# Title 전처리 및 df['noun'] 생성 -> Title 에서 명사만 생성
n_ = []
title_rename = []

for i in tqdm(range(len(df))) : 
    # 기사 제목 앞 뒤로 괄호/대괄호 지우기
    title_rename.append(re.sub(r'[\(\[].*?[\)\]]', '', df.iloc[i]['Title']))
    
    # 바뀌어진 기사 제목에서 okt로 명사만 뽑기
    n_.append(' '.join(okt.nouns(df.iloc[i]['Title'])))

df['nouns'] = n_
df['Title'] = title_rename
df = df[df['nouns'] != ''] 
df.head(3)

100%|█████████████████████████████████████████████████████████████████████████| 429241/429241 [08:54<00:00, 803.23it/s]


,Date,Title,주가의 날짜,등락률(y),updown,nouns
0,2021-01-04,비올 일본 최대 병원체인과 실펌엑스 총판계약 체결,2021-01-05,2.24,1,비올 일본 최대 병원체 인과 실펌 엑스 판 계약 체결
1,2021-01-04,환율 하락 전환 1086 2 감소0 1원,2021-01-05,2.24,1,환율 하락 전환 감소
2,2021-01-04,코스피 1 03p 0 04 오른 2874 50 출발 원 달러 환율 1 2원 ...,2021-01-05,2.24,1,코스피 오른 출발 원 달러 환율 오른 출발


In [17]:
#df.to_csv('./[Model1_multi]cleaned_df.csv', encoding = 'utf-8-sig')

In [18]:
# 윗부분 건너뛰고 여기부터 테스트하고 싶을 때
#df = pd.read_csv('./[Model1_multi]cleaned_df.csv', index_col = 0)

In [19]:
vocab = {}
cnt = 0

for i in tqdm(df['nouns']) : 
    i = i.split(' ')   
    
    for j in range(len(i)) : 
        if i[j] in vocab or len(i[j]) <= 1:    
            cnt += 1
        else :    
            vocab[i[j]] = 0     

print(len(vocab))
print(vocab)

100%|██████████████████████████████████████████████████████████████████████| 427673/427673 [00:01<00:00, 277273.35it/s]

35683
{'비올': 0, '일본': 0, '최대': 0, '병원체': 0, '인과': 0, '실펌': 0, '엑스': 0, '계약': 0, '체결': 0, '환율': 0, '하락': 0, '전환': 0, '감소': 0, '코스피': 0, '오른': 0, '출발': 0, '달러': 0, '증시': 0, '사상': 0, '최고': 0, '시작': 0, '개장': 0, '안정': 0, '시장': 0, '운영': 0, '방점': 0, '바이오': 0, '아벨': 0, '매각': 0, '자본': 0, '이득': 0, '대웅제약': 0, '코로나': 0, '치료': 0, '시험': 0, '주가': 0, '상승': 0, '코스닥': 0, '포인트': 0, '신년사': 0, '영채': 0, '투자': 0, '증권': 0, '대표': 0, '데이터': 0, '산이': 0, '경쟁력': 0, '케이피': 0, '에스': 0, '올해': 0, '사업': 0, '시너지': 0, '효과': 0, '기관': 0, '매도': 0, '자산운용': 0, '조직개편': 0, '단행': 0, '조직': 0, '확대': 0, '대체': 0, '신설': 0, '가치투자': 0, '대가': 0, '방천': 0, '주식': 0, '펀드': 0, '가계': 0, '연금': 0, '자산': 0, '변화': 0, '개인': 0, '매수': 0, '스팟': 0, '외국인': 0, '중립': 0, '윤관석': 0, '정무': 0, '위원장': 0, '자본시장': 0, '질서': 0, '유지': 0, '삼성': 0, '전자': 0, '반도체': 0, '업황': 0, '회복': 0, '기대': 0, '목표': 0, '하이': 0, '성수': 0, '금융': 0, '사모펀드': 0, '사태': 0, '신뢰': 0, '만전': 0, '손병두': 0, '거래소': 0, '이사장': 0, '공매도': 0, '적법': 0, '사전': 0, '사후': 0, '관리': 0, '강화': 0, '증가': 0, '속보': 0,

In [20]:
df['updown'].value_counts()

 0    169318
 1    141094
-1    117261
Name: updown, dtype: int64

# vocab score 생성

In [21]:
up = len(df[df['updown'] == 1])
mid = len(df[df['updown'] == 0])
down = len(df[df['updown'] == -1])

up_ratio = up / (up + mid + down)
mid_ratio = mid / (up + mid + down)
down_ratio = down / (up + mid + down)

print('up_ratio : ', up_ratio)
print('mid_ratio : ', mid_ratio)
print('down_ratio : ', down_ratio)

up_ratio :  0.32991093662681537
mid_ratio :  0.3959052827744562
down_ratio :  0.27418378059872844


In [22]:
for i,w in tqdm(enumerate(df['nouns'])) :
    w = w.split(' ')
    
    if df.iloc[i]['updown'] == 1 :
        for j in range(len(w)) :
            noun = w[j]
            if len(noun) <= 1 :
                continue
            vocab[noun] += down_ratio
            
    elif df.iloc[i]['updown'] == 0 : 
        for j in range(len(w)) : 
            noun = w[j] 
            if len(noun) <= 1 : 
                continue
            vocab[noun] += mid_ratio  # 더하는 게 맞을까?  
    else :
        for j in range(len(w)) :
            noun = w[j]
            if len(noun) <= 1 :
                continue
            vocab[noun] -= up_ratio

427673it [01:30, 4732.27it/s]


In [23]:
vocab

{'비올': 6.096795916506303,
 '일본': 109.09647324006839,
 '최대': 1500.071281095726,
 '병원체': 0.27418378059872844,
 '인과': 2.6144227014564874,
 '실펌': 0.5483675611974569,
 '엑스': 82.75071842271967,
 '계약': 1471.9016047308355,
 '체결': 897.5448952821741,
 '환율': 1217.0549742444432,
 '하락': 4164.125598297248,
 '전환': 995.9352846684793,
 '감소': 5529.088591048276,
 '코스피': 5398.043547288322,
 '오른': 776.9277976398063,
 '출발': 1623.6671358726467,
 '달러': 1413.8595562498679,
 '증시': 2539.5221652997707,
 '사상': 723.7538937459418,
 '최고': 1094.3558630075463,
 '시작': 334.75225698138456,
 '개장': 587.0853642853301,
 '안정': 180.65196306523754,
 '시장': 1267.6977644135663,
 '운영': 160.9958356033688,
 '방점': 13.83430331117464,
 '바이오': 1453.0080014404705,
 '아벨': 2.589375527564284,
 '매각': 545.4404720428896,
 '자본': 124.52781447507725,
 '이득': 1.269792575168411,
 '대웅제약': 47.91259677370314,
 '코로나': 1315.253640047505,
 '치료': 627.3023431453529,
 '시험': 63.27331161892357,
 '주가': 2011.1156982088883,
 '상승': 7850.834172365616,
 '코스닥': 2811.32

# vocab 정규화

In [24]:
robustScaler = RobustScaler()

In [25]:
# scale 할 리스트
vocab_v = [v for v in vocab.values()]
vocab_v_arr = np.array(vocab_v)

In [26]:
scaled_list = robustScaler.fit_transform(vocab_v_arr.reshape(-1,1))

In [27]:
scaled_list = np.squeeze(scaled_list, axis = 1)

In [28]:
for idx, v in tqdm(enumerate(vocab_v)) :
    vocab_v[idx] = list(scaled_list)[idx]

for i, k in tqdm(enumerate(vocab.keys())) :
    vocab[k] = vocab_v[i]

35683it [00:38, 929.52it/s]
35683it [00:00, 1703743.52it/s]


In [29]:
vocab

{'비올': 2.739019999734513,
 '일본': 53.585522484623176,
 '최대': 740.2498486443225,
 '병원체': -0.13535271802584783,
 '인과': 1.0199223973594556,
 '실펌': 0.0,
 '엑스': 40.579758558019634,
 '계약': 726.3436932416565,
 '체결': 442.80853455904617,
 '환율': 600.5368929971709,
 '하락': 2055.378672144805,
 '전환': 491.37963020145446,
 '감소': 2729.2020860247585,
 '코스피': 2664.5107945924447,
 '오른': 383.2650707894781,
 '출발': 801.2637964579145,
 '달러': 697.6908349123573,
 '증시': 1253.3819607049318,
 '사상': 357.0154056766719,
 '최고': 539.9656288290129,
 '시작': 164.98205951151732,
 '개장': 289.548041667412,
 '안정': 88.90938451744708,
 '시장': 625.537056328624,
 '운영': 79.20600136321107,
 '방점': 6.558693994769918,
 '바이오': 717.0167354140684,
 '아벨': 1.0075576868803284,
 '매각': 268.9897528269889,
 '자본': 61.203310722796154,
 '이득': 0.3561364435181938,
 '대웅제약': 23.38167905888755,
 '코로나': 649.0133429678149,
 '치료': 309.4014311990807,
 '시험': 30.964602091681336,
 '주가': 992.5304566990881,
 '상승': 3875.347836548637,
 '코스닥': 1387.5604375440023,
 '포인

# sent_score 설정

In [30]:
total = []

for i,w  in tqdm(enumerate(df['nouns'])) :
    sent_score = 0
    w = w.split(' ')
    
    for j in w :
        if len(j) <= 1 :
            continue
        elif j not in vocab :
            continue
        else :
            sent_score +=  vocab[j]
            
    total.append(sent_score / len(w))

df['sent_score'] = total
df.head()

427673it [00:01, 216026.38it/s]


,Date,Title,주가의 날짜,등락률(y),updown,nouns,sent_score
0,2021-01-04,비올 일본 최대 병원체인과 실펌엑스 총판계약 체결,2021-01-05,2.24,1,비올 일본 최대 병원체 인과 실펌 엑스 판 계약 체결,200.719095
1,2021-01-04,환율 하락 전환 1086 2 감소0 1원,2021-01-05,2.24,1,환율 하락 전환 감소,1469.124320
2,2021-01-04,코스피 1 03p 0 04 오른 2874 50 출발 원 달러 환율 1 2원 ...,2021-01-05,2.24,1,코스피 오른 출발 원 달러 환율 오른 출발,791.474532
3,2021-01-04,韓증시 사상최고치 시작 개장식선 안정적 시장운영에 방점,2021-01-05,2.24,1,증시 사상 최고 치 시작 개장 선 안정 시장 운영 방점,309.554930
4,2021-01-04,SK바이오팜 아벨 지분 매각으로 5천500만 달러 자본이득,2021-01-05,2.24,1,바이오 아벨 분 매각 달러 자본 이득,249.466333


In [31]:
desc = df['sent_score'].describe()
desc

count    427673.000000
mean        408.715578
std         368.245406
min          -4.085983
25%         145.625767
50%         307.069524
75%         563.017385
max        3875.347837
Name: sent_score, dtype: float64

In [32]:
q3 = desc['75%']
q1 = desc['25%']

In [33]:
df['sent_label'] = 0

In [34]:
df.loc[df.query(f'sent_score >= {q3}').index, 'sent_label'] = 1 
df.loc[df.query(f'sent_score <= {q1}').index, 'sent_label'] = -1 

In [35]:
df.head()

,Date,Title,주가의 날짜,등락률(y),updown,nouns,sent_score,sent_label
0,2021-01-04,비올 일본 최대 병원체인과 실펌엑스 총판계약 체결,2021-01-05,2.24,1,비올 일본 최대 병원체 인과 실펌 엑스 판 계약 체결,200.719095,0
1,2021-01-04,환율 하락 전환 1086 2 감소0 1원,2021-01-05,2.24,1,환율 하락 전환 감소,1469.124320,1
2,2021-01-04,코스피 1 03p 0 04 오른 2874 50 출발 원 달러 환율 1 2원 ...,2021-01-05,2.24,1,코스피 오른 출발 원 달러 환율 오른 출발,791.474532,1
3,2021-01-04,韓증시 사상최고치 시작 개장식선 안정적 시장운영에 방점,2021-01-05,2.24,1,증시 사상 최고 치 시작 개장 선 안정 시장 운영 방점,309.554930,0
4,2021-01-04,SK바이오팜 아벨 지분 매각으로 5천500만 달러 자본이득,2021-01-05,2.24,1,바이오 아벨 분 매각 달러 자본 이득,249.466333,0


In [36]:
print(df['sent_label'].value_counts())

 0    213834
 1    106920
-1    106919
Name: sent_label, dtype: int64


In [37]:
print(df['updown'].value_counts())

 0    169318
 1    141094
-1    117261
Name: updown, dtype: int64


In [38]:
print(accuracy_score(df['updown'], df[ 'sent_label']))

#print(accuracy_score(df.loc[df.query('sent_label == 1').index, 'updown'],
#               df.loc[df.query('sent_label == 1').index, 'sent_label']))

#print(accuracy_score(df.loc[df.query('sent_label == -1').index, 'updown'],
#               df.loc[df.query('sent_label == -1').index, 'sent_label']))

0.3505622286185941


# 트레인 테스트 분리¶

In [39]:
# X : title, y : price
def data_split(X, y) :
    
    # 학습셋, 테스트셋 분리
    X_list = X.tolist()
    y_list = y.tolist()
    
    X_train, X_test, y_train, y_test = train_test_split(X_list, y_list, shuffle = True, test_size = 0.2)
    
    return X_train, X_test, y_train, y_test

In [40]:
X_train, X_test, y_train, y_test = data_split(df['Title'], df['sent_label'])

In [41]:
print(len(X_train))
print(len(X_test))

342138
85535


# 토큰화 및 로지스틱 회귀

In [42]:
# 형태소 분석을 위한 함수
def tokenizer(text):
    okt = Okt()
    return okt.morphs(text)

In [43]:
tokenize_X_train = TfidfVectorizer(tokenizer = tokenizer).fit_transform(X_train)
tokenize_X_test = TfidfVectorizer(tokenizer = tokenizer).fit_transform(X_test)

# 로지스틱 회귀 그리드 서치

In [44]:
def parameter_serach(penalty, solvers) :
    best_score = 0
    Cs = [0.001, 0.01, 0.1, 1, 10, 100]
        
    kfold = KFold(3, shuffle = True)

    for s in solvers :
        scores_mean = []
        for C in tqdm(Cs) :
            lm = LogisticRegression(penalty = penalty, C = C, solver = s, multi_class = 'multinomial')
            scores = cross_val_score(lm, tokenize_X_train, y_train, cv = kfold)

            score = np.mean(scores)
            scores_mean.append(score)

            if score > best_score:
                best_score = score
                best_parameters = {'penalty' : penalty, 'C': C, 'solver': s, 'best_score' : best_score}

    return best_parameters 

In [45]:
# saga : 쓸 수 있지만 C값이 1 이상부터 시간이 너무 오래 걸림(몇분 단위)
#l1_best_parameters = parameter_serach('l1', ['saga'])

In [46]:
l2_best_parameters = parameter_serach('l2', ['lbfgs', 'newton-cg',  'sag', 'saga'])

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [02:03<00:00, 20.62s/it]


### 로지스틱 회귀의 최적 파라미터 확인

In [47]:
# print(f"penalty\t\t:\t{l1_best_parameters['penalty']}")
# print(f"C\t\t:\t{l1_best_parameters['C']}")
# print(f"solver\t\t:\t{l1_best_parameters['solver']}")
# print(f"best_score\t:\t{l1_best_parameters['best_score']}")

In [48]:
print(f"penalty\t\t:\t{l2_best_parameters['penalty']}")
print(f"C\t\t:\t{l2_best_parameters['C']}")
print(f"solver\t\t:\t{l2_best_parameters['solver']}")
print(f"best_score\t:\t{l2_best_parameters['best_score']}")

penalty		:	l2
C		:	100
solver		:	saga
best_score	:	0.9497834207249706


In [49]:
# if l1_best_parameters['best_score'] > l2_best_parameters['best_score'] :
#     best_param = l1_best_parameters
# else :
#     best_param = l2_best_parameters

In [50]:
best_param = l2_best_parameters
best_param 

{'penalty': 'l2', 'C': 100, 'solver': 'saga', 'best_score': 0.9497834207249706}

# 파이프 라인

In [51]:
# lowercase : 소문자로 반환 여부 lowercase = False, 
# tokenizer : 토크나이저 지정
tfidf = TfidfVectorizer(tokenizer = tokenizer)

In [52]:
logistic = LogisticRegression(penalty = best_param['penalty'],
                              C = best_param['C'],
                              solver = best_param['solver'],
                              multi_class = 'multinomial' )

In [53]:
pipeline = Pipeline([('tfidf',tfidf), ('classfier', logistic)], verbose = True) 

In [54]:
pipeline.fit(X_train, y_train)

[Pipeline] ............. (step 1 of 2) Processing tfidf, total=12.2min
[Pipeline] ......... (step 2 of 2) Processing classfier, total=  35.0s


Pipeline(steps=[('tfidf',
                 TfidfVectorizer(tokenizer=<function tokenizer at 0x00000241DFDB5D30>)),
                ('classfier',
                 LogisticRegression(C=100, multi_class='multinomial',
                                    solver='saga'))],
         verbose=True)

### 교차검증

In [55]:
score = cross_val_score(pipeline, X_train, y_train, cv = KFold(3, shuffle = True), scoring='f1_micro')

[Pipeline] ............. (step 1 of 2) Processing tfidf, total= 5.8min
[Pipeline] ......... (step 2 of 2) Processing classfier, total=  19.5s
[Pipeline] ............. (step 1 of 2) Processing tfidf, total= 5.2min
[Pipeline] ......... (step 2 of 2) Processing classfier, total=  17.6s
[Pipeline] ............. (step 1 of 2) Processing tfidf, total= 5.1min
[Pipeline] ......... (step 2 of 2) Processing classfier, total=  19.7s


In [56]:
print(score)
print(f"모델의 정확도\t:\t{score.mean()}")

[0.94978342 0.94945022 0.95000263]
모델의 정확도	:	0.9497454243609305


In [58]:
y_pred = pipeline.predict(X_test)

In [59]:
y_proba = pipeline.predict_proba(X_test)

# 평가

In [60]:
print(f'accuracy \t:\t {accuracy_score(y_test, y_pred)}')

#'micro', 'macro', 'weighted'
print(f'f1_score \t:\t {f1_score(y_test, y_pred, average = "micro")}')
print(f'f1_score \t:\t {f1_score(y_test, y_pred, average = "macro")}')
print(f'f1_score \t:\t {f1_score(y_test, y_pred, average = "weighted")}')

print(f'R2 \t\t:\t {r2_score(y_test, y_pred)}')

print(f'mse \t\t:\t {mean_squared_error(y_test, y_pred)}')

print(f'rmse \t\t:\t {mean_squared_error(y_test, y_pred, squared=False)}')

print(f'log_loss\t:\t {log_loss(y_test, y_proba)}')

accuracy 	:	 0.959303209212603
f1_score 	:	 0.959303209212603
f1_score 	:	 0.9592475406616435
f1_score 	:	 0.9593006877052767
R2 		:	 0.9182593108832237
mse 		:	 0.04080201087274215
rmse 		:	 0.201995076357673
log_loss	:	 0.11426404543775542


In [61]:
print(confusion_matrix(y_pred, y_test))
print(classification_report(y_pred, y_test))

[[20323   918     1]
 [ 1012 41151   778]
 [    2   770 20580]]
              precision    recall  f1-score   support

          -1       0.95      0.96      0.95     21242
           0       0.96      0.96      0.96     42941
           1       0.96      0.96      0.96     21352

    accuracy                           0.96     85535
   macro avg       0.96      0.96      0.96     85535
weighted avg       0.96      0.96      0.96     85535



# 모델 저장 및 사용

In [62]:
def save_model(model) :
    with open('./[Model1]pipe_multi.dat', 'wb') as fp :     # 쓰기
        pickle.dump(model, fp)
    print('저장완료')     # 학습된 모델 저장 완료

In [63]:
save_model(pipeline)

저장완료


In [64]:
with open('./[Model1]pipe_multi.dat','rb') as fp :     # 읽기
    pipeline = pickle.load(fp)

# Inference

In [65]:
# 모델 사용 함수

def model_prediction():  
    # 객체를 복원, 저장된 모델 불러오기
    with open('./[Model1]pipe_multi.dat','rb') as fp :     # 읽기
        pipe = pickle.load(fp)
    while True :
        text = input('뉴스 타이틀을 입력해주세요(종료를 원하시면 "q"를 입력해주세요) : \n')
        example = [text]
        # 예측 정확도
        r1 = np.max(pipe.predict_proba(example) * 100)     # 확률값을 구해서 *100
        
        # 예측 결과 
        r2 = pipe.predict(example)[0]     # 긍정(1), 중립(0), 부정(-1)

        if text == 'q' :
            print("예측을 종료합니다.")
            break
        if r2 == 1 :
            print(f'\n{stock_name} 주가가 상승할 것으로 예상됩니다.')
        elif r2 == -1 :
            print(f'\n{stock_name} 주가가 하락할 것으로 예상됩니다.')
        else : 
            print(f'\n모르겠어요.....')
        print('확률 : %.3f' % r1)
        print('------------------------------------------------\n')

In [66]:
model_prediction()

뉴스 타이틀을 입력해주세요(종료를 원하시면 "q"를 입력해주세요) : 
[줌인 이종목] 현대중공업, 실적 증가 기대에 껑충

모르겠어요.....
확률 : 98.162
------------------------------------------------

뉴스 타이틀을 입력해주세요(종료를 원하시면 "q"를 입력해주세요) : 
닷새만에 반등했지만…불안한 개미들, 7100억 차익실현

모르겠어요.....
확률 : 98.962
------------------------------------------------

뉴스 타이틀을 입력해주세요(종료를 원하시면 "q"를 입력해주세요) : 
주식거래 뚝…"증권사, 2분기 실적 쇼크"

모르겠어요.....
확률 : 53.858
------------------------------------------------

뉴스 타이틀을 입력해주세요(종료를 원하시면 "q"를 입력해주세요) : 
20년 만의 '1유로=1달러'.. 경기침체 공포에 달러 초강세

모르겠어요.....
확률 : 99.994
------------------------------------------------

뉴스 타이틀을 입력해주세요(종료를 원하시면 "q"를 입력해주세요) : 
블랙록 '극심한 변동성' 경고.."지금은 저가매수도 하지 말라"

삼성SDI 주가가 하락할 것으로 예상됩니다.
확률 : 100.000
------------------------------------------------

뉴스 타이틀을 입력해주세요(종료를 원하시면 "q"를 입력해주세요) : 
'현대차 효과' 부품株에도 볕드나.. 미래형 자동차 양산 기대감 호재 [현대차, 29년만에 국내공장 신설]

모르겠어요.....
확률 : 99.823
------------------------------------------------

뉴스 타이틀을 입력해주세요(종료를 원하시면 "q"를 입력해주세요) : 
q
예측을 종료합니다.


```
[줌인 이종목] 현대중공업, 실적 증가 기대에 껑충
닷새만에 반등했지만…불안한 개미들, 7100억 차익실현
주식거래 뚝…"증권사, 2분기 실적 쇼크"
'현대차 효과' 부품株에도 볕드나.. 미래형 자동차 양산 기대감 호재 [현대차, 29년만에 국내공장 신설]
2차전지株, 실적 부진해도 상승하는 까닭
블랙록 '극심한 변동성' 경고.."지금은 저가매수도 하지 말라"
20년 만의 '1유로=1달러'.. 경기침체 공포에 달러 초강세

```